In [304]:
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import numpy as np
from collections import Counter
import nltk
from nltk.stem.porter import PorterStemmer

ModuleNotFoundError: No module named 'chap_3'

In [234]:
datasets_folder = "spam_datasets"

In [235]:
def spam_ham_string(x, spam_or_ham):
    if (spam_or_ham != "spam" and spam_or_ham!="ham"):
        print("Error in input string")
        return False
    if spam_or_ham in x:
        return True
    return False

In [236]:
def split_spam_ham_name(all_folders):
    spam_folders = list(filter((lambda x: spam_ham_string(x, "spam")), all_folders))
    ham_folders = list(filter((lambda x: spam_ham_string(x, "ham")), all_folders))
    return ([spam_folders, ham_folders])

In [237]:
def extract_spam_ham_name(datasets_folder):
    current_folder_path = os.path.abspath(os.getcwd()) + "/"
    dataset_path = current_folder_path + datasets_folder + "/"
    all_folders = os.listdir(dataset_path)
    
    spam_folders, ham_folders = split_spam_ham_name(all_folders)
    
    spam_folders_path = [dataset_path + folder + "/" for folder in spam_folders]
    ham_folders_path = [dataset_path + folder + "/" for folder in ham_folders]
    
    return [spam_folders_path, ham_folders_path]

In [238]:
spam_folders_path, ham_folders_path = extract_spam_ham_name(datasets_folder)

In [239]:
spam_folders_path

['/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20030228_spam_2/',
 '/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20021010_spam/',
 '/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20030228_spam/']

In [240]:
ham_folders_path

['/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20030228_easy_ham_2/',
 '/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20030228_hard_ham/',
 '/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20030228_easy_ham/',
 '/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20021010_easy_ham/',
 '/home/hugo/Machine_learning/Machine_learning_course/chap_3/spam_datasets/20021010_hard_ham/']

In [241]:
def separate_header_content(raw_text):
    separator_pattern = re.compile(r"\n\n")
    mail = separator_pattern.split(raw_text, maxsplit=1)
    if len(mail)==1:
        print(mail)
    heading = mail[0]
    content = mail[1]
    header_in_content = False
    while(content != "" and header_in_content):
        header_in_content = False
        for pattern in heading_pattern:
            if pattern.match(content) != None:
                mail = separator_pattern.split(content, maxsplit=1)
                header_in_content = True
                content = mail[1]
                heading = heading + mail[0]
    
    return (heading, content)
            

In [242]:
def extract_txt(filepath):
    try:
        f = open(filepath, "r", encoding='utf8', errors='replace')#, 
        text = f.read()
    except:
        return (False, filepath)
    return (True, text)

Patterns in the heading
---

In [243]:

received_pattern = re.compile("^Received:.*\n\t.*\n\t.*\n|^Received:.*\n\t.*\n|^Received:.*\n",
                              flags=re.M|re.X)
messageId_pattern = re.compile("^Message-ID:.*\n",
                              flags=re.M|re.X)
returnpath_pattern = re.compile("^Return-Path:.*\n",
                              flags=re.M|re.X)
from_pattern = re.compile("^From[:\s].*\n",
                         flags=re.M|re.X)
deliver_pattern = re.compile("^Delivered-To:.*\n", 
                            flags=re.M|re.X)
subject_pattern = re.compile("^Subject:.*\n", 
                            flags=re.M|re.X)
date_pattern = re.compile("^Date:.*\n", 
                            flags=re.M|re.X)
content_type_pattern = re.compile("^Content-Type:.*\n", 
                            flags=re.M|re.X)
content_transfer_pattern = re.compile("^Content-Transfer-Encoding:.*\n", 
                            flags=re.M|re.X)
content_pattern = re.compile("^Content-.*:.*\n",
                            flags=re.M|re.X)
mime_pattern = re.compile("^MIME-Version:.*\n", 
                            flags=re.M|re.X)
bcc_pattern = re.compile("^Bcc:.*\n", 
                            flags=re.M|re.X)
importance_pattern = re.compile("^Importance:.*\n", 
                            flags=re.M|re.X)
to_pattern = re.compile("^To:.*\n", 
                            flags=re.M|re.X)
xmailer_pattern = re.compile("^X-Mailer:.*\n", 
                            flags=re.M|re.X)
xpriority_pattern = re.compile("^X-Priority:.*\n", 
                            flags=re.M|re.X)
x_spam_pattern = re.compile("^X-Spam:.*\n",
                           flags=re.M|re.X)
x_pattern = re.compile("^X-.*:.*\n", 
                            flags=re.M|re.X)
replyto_pattern = re.compile("^Reply-To:.*\n", 
                            flags=re.M|re.X)
list_pattern = re.compile("^List-.*:.*\n", 
                            flags=re.M|re.X)
errors_pattern = re.compile("^Errors:.*\n", 
                            flags=re.M|re.X)
hyphen_to_pattern = re.compile("^[\w\-]+-To:.*\n", 
                            flags=re.M|re.X)

heading_pattern = [
    received_pattern, 
    messageId_pattern,
    returnpath_pattern,
    from_pattern,
    deliver_pattern,
    subject_pattern,
    date_pattern,
    content_type_pattern,
    content_transfer_pattern,
    mime_pattern,
    bcc_pattern,
    importance_pattern,
    to_pattern,
    xmailer_pattern,
    xpriority_pattern,
    x_spam_pattern,
    x_pattern,
    replyto_pattern,
    list_pattern,
    errors_pattern,
    hyphen_to_pattern,
    ]

Patterns in the content
----

In [244]:
HTML_pattern = re.compile("\<.*?\>", re.S)
URL_pattern = re.compile("http.*\s", re.I | re.X)
mail_pattern = re.compile("\w+@[\w\.]+\W", re.I)
arrow_pattern = re.compile("^\>", re.M)
price_pattern = re.compile("[\$£]\s{0,2}\d+[,\.]\d+ | [\$£]\s{0,2}\d+ | \d+\s{0,2}[$£] | \d+[,\.]\d+\s{0,2}[$£]", re.X)
one_two_letter_word_pattern = re.compile(r"\b[a-zA-Z]{1,2}\b", re.X | re.M)

punctuation_pattern = re.compile(r"[\?\!]", re.X)
hour_date_pattern = re.compile("\d{4}\s*[a-zA-Z]{2,8}\s*\d{1,2} | \d{1,2}\s*[a-zA-Z]{2,8}\s*\d{4} | \d{1,2}:\d{2}:\d{2}", re.X)

maj_pattern = re.compile("[A-Z]", re.X)
number_pattern = re.compile("\d+", re.X)

General class to process heading
---

In [245]:
class HeadPattern():
    def __init__(self, pattern):
        self.pattern = pattern
        self.pattern_list = []
    
    @property
    def pattern_cat(self):
        return 0    
    
    def find(self, header):
        self.pattern_list = self.pattern.findall(header)
        
    def isNone(self):
        return (self.pattern_list == [])
    
    def display_pattern(self):
        print(self.pattern_list)
    
    def len_list_pattern(self):
        return (len(self.pattern_list))
    
    def processing(self, header):
        pass

General class to process content
----

In [246]:
class ContentPattern():
    def __init__(self, pattern):
        self.pattern = pattern
        self.pattern_list = []
    
    @property
    def pattern_cat(self):
        return 0    
    
    def find(self, content):
        self.pattern_list = self.pattern.findall(content)
        
    def isNone(self):
        return (self.pattern_list == [])
    
    def display_pattern(self):
        print(self.pattern_list)
    
    def len_list_pattern(self):
        return (len(self.pattern_list))
    
    def count(self, content):
        self.find(content)
        return (self.len_list_pattern())
    
    def replace(self, content):
        new_content = self.pattern.sub("", content)
        return (new_content)
    
    def count_replace(self, content):
        pattern_nb = 0
        new_content, pattern_nb = self.pattern.subn("", content)
        return (new_content, pattern_nb)
    
    def processing(self, content):
        pass

Subclasses for processing
----

In [247]:
class FromPattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, from_pattern)
    
    def processing(self, header):
        self.find(header)
        name = ''
        domain = ''
        extension = ''
        if (self.isNone() == False):
            length = self.len_list_pattern()
            for i in range(length):
                mail_pattern1 = re.compile("(From:\s)<?(?P<name>.*)@(?P<domain>.*)\.(?P<extension>[\w\.]*)>?", re.I)
                #mail_pattern2 = re.compile("(From:\s)(?P<name>.*)@(?P<domain>.*)\.(?P<extension>.*)", re.I)
                mail_address = mail_pattern1.search(self.pattern_list[i])
                #if (mail_address == None):
                #    mail_address = mail_pattern2.search(self.pattern_list[i])
                if (mail_address != None):
                    name = mail_address.group('name')
                    domain = mail_address.group('domain')
                    extension = mail_address.group('extension')
        len_name = len(name)
        number_in_name = re.findall("\d", name)
        if (number_in_name != None):
            len_number_in_name = len(re.findall("\d", name))
        else:
            len_number_in_name = 0
        return ([len_name, len_number_in_name, domain, extension])
    
    @property
    def pattern_cat(self):
        return ("from")

In [248]:
class DatePattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, date_pattern)
    
    def processing(self, header):
        self.find(header)
        hour = np.nan
        if (self.isNone() == False):
            hour_patt = re.compile("(?P<hour>\d{1,2}):(?P<minute>\d{2}):(?P<second>\d{2})")
            hour_search = hour_patt.search(self.pattern_list[0])
            hour = int(hour_search.group('hour')) if (hour_search!=None) else np.nan
        return (hour)
    @property
    def pattern_cat(self):
        return("hour")

In [249]:
class ContentTypePattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, content_type_pattern)
    
    def processing(self, header):
        self.find(header)
        content_type = np.nan
        if (self.isNone() == False):
            content_type_patt = re.compile("""(Content-Type:)\s(?P<type>.*)[;\n]""", re.X)
            content_type_match = content_type_patt.search(self.pattern_list[0])
            content_type = content_type_match.group('type') if (content_type_match!=None) else np.nan
        return (content_type)
    
    @property
    def pattern_cat(self):
        return("content-type")

In [250]:
class ImportancePattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, importance_pattern)
    
    def processing(self, header):
        self.find(header)
        importance = np.nan
        if (self.isNone() == False):
            importance = self.pattern_list[0][12:]
        return (importance)
    @property
    def pattern_cat(self):
        return("importance")

In [251]:
class XSpamPattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, x_spam_pattern)
    
    def processing(self, header):
        self.find(header)
        x_spam_word = np.nan
        if (self.isNone() == False):
            x_spam_word = self.pattern_list[0][8:]
        return (x_spam_word)
    
    @property
    def pattern_cat(self):
        return("x-spam")

In [252]:
class X_Pattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, x_pattern)
    
    def processing(self, header):
        self.find(header)
        return (self.len_list_pattern())
    
    @property
    def pattern_cat(self):
        return("x-*")

In [253]:
class HTMLPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, HTML_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    @property
    def pattern_cat(self):
        return("html")

In [254]:
class URLPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, URL_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    
    @property
    def pattern_cat(self):
        return("url")

In [255]:
class MailPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, mail_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    
    @property
    def pattern_cat(self):
        return("mail")

In [256]:
class ArrowPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, arrow_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    
    @property
    def pattern_cat(self):
        return("> count")

In [257]:
class PricePattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, price_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    
    @property
    def pattern_cat(self):
        return("price")

In [258]:
class PunctuationPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, punctuation_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    
    @property
    def pattern_cat(self):
        return("? or !")

In [259]:
class NumberPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, number_pattern)
    
    def processing(self, content):
        return self.count_replace(content)
    
    @property
    def pattern_cat(self):
        return("Number count")

In [260]:
class HourDatePattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, hour_date_pattern)
    
    def processing(self, content):
        return self.replace(content)

In [261]:
class OneTwoLetterWordPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, one_two_letter_word_pattern)
    
    def processing(self, content):
        return self.replace(content)

In [262]:
class MajPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, maj_pattern)
    
    def processing(self, content):
        return self.count(content)
    
    @property
    def pattern_cat(self):
        return("Majuscule count")

In [263]:
word_pattern = re.compile("\\b[\w\+\-\=\&\*]+\\b", re.I | re.X)

class WordPattern(ContentPattern):
    def __init__(self):
        ContentPattern.__init__(self, word_pattern)
    
    def processing(self, content):
        self.find(content)
        number_of_word = self.len_list_pattern()
        len_word_list = list(map(len, self.pattern_list))
        if len(len_word_list) != 0:
            mean_len_word = round(sum(len_word_list)/len(len_word_list), 4)
            longuest_word = max(len_word_list)
        else:
            mean_len_word = 0
            longuest_word = 0
        return (content, [number_of_word, mean_len_word, longuest_word])
    @property
    def pattern_cat(self):
        return("content_attr")
        

In [264]:
def isolate_all_word(content):
    pattern = re.compile("\w+", re.I | re.X | re.M)
    new_content = pattern.findall(content)
    return new_content

Subject is treated differently from other heading pattern as precious info could be extract from these
---

In [265]:
class SubjectPattern(HeadPattern):
    def __init__(self):
        HeadPattern.__init__(self, subject_pattern)
    
    def processing(self, header):
        self.find(header)
        number_of_maj = 0
        number_of_number = 0
        number_of_char = 0
        number_of_price = 0
        number_of_special_char = 0
        word_list = []
        if (self.isNone() == False):
            # Extract info on number of char in function of categories (maj, punctuation)
            number_of_maj = len(re.findall("[A-Z]", self.pattern_list[0])) - 1 #Subtract "S" of "Subject"
            number_of_char = len(self.pattern_list[0]) - 8 #Subtract len("Subject:"))
            number_of_special_char = len(re.findall("[\!\?\$]", self.pattern_list[0]))
            
            # Extract info on number of char in function of categories (price,number)
            # And replace by "" those pattern
            new_subject, number_of_price = PricePattern().processing(self.pattern_list[0])
            new_subject, number_of_number = NumberPattern().processing(new_subject)
            # Replace all one or two letters word by ""
            new_subject = OneTwoLetterWordPattern().processing(new_subject)
            
            # Extact all remaining words except Subject of course
            new_subject = new_subject[8:]
            word_list = isolate_all_word(new_subject)
        
        return ([number_of_maj, 
                 number_of_char, 
                 number_of_special_char, 
                 number_of_price, 
                 number_of_number, 
                 word_list])
    
    @property
    def pattern_cat(self):
        return("subject")

In [266]:
string = "Subject: jzeidze dzejdiude09909n dedzi 999jdeiz i9 JJ"
subj_patt = SubjectPattern()
subj_extract = subj_patt.processing(string)
subj_extract

[2, 45, 0, 0, 3, ['jzeidze', 'dzejdiuden', 'dedzi', 'jdeiz']]

In [267]:
important_heading_pattern = [
    FromPattern(),
    SubjectPattern(),
    DatePattern(),
    ContentTypePattern(),
    ImportancePattern(),
    XSpamPattern(),
    X_Pattern(),
]

In [268]:
def process_heading(heading):
    attributes = []
    for important_pattern in important_heading_pattern:
        attributes.append(important_pattern.processing(heading))
    return attributes  #heading_dataframe.loc[len(heading_dataframe.index)] = 

In [269]:
count_replace_content_pattern_list = [
    HTMLPattern(),
    URLPattern(),
    MailPattern(),
    PricePattern(),
    ArrowPattern(),
    PunctuationPattern(),
    WordPattern(),
    NumberPattern(),
]

replace_content_list = [
    OneTwoLetterWordPattern(),
    HourDatePattern(),
]

count_content_list = [
    MajPattern(),
]

In [270]:
def process_content(content):
    attributes = []
    new_content = content
    for count_repl_patt in count_replace_content_pattern_list:
        new_content, count = count_repl_patt.processing(new_content)
        attributes.append(count)
    for repl_patt in replace_content_list:
        new_content = repl_patt.processing(new_content)
    for count_patt in count_content_list:
        count = count_patt.processing(new_content)
        attributes.append(count)
    word_list = isolate_all_word(new_content)
    attributes.append(word_list)
    return attributes

In [271]:
def create_dataframe_cat():
    # Create Dataframe
    mail_dataframe = pd.DataFrame()
    
    # Add a spam or not cat
    mail_dataframe['spam']=[]
    
    # Add Heading Attributes
    for important_pattern in important_heading_pattern:
        mail_dataframe[important_pattern.pattern_cat]=[]
    
    # Add Content Attributes
    for count_repl_patt in count_replace_content_pattern_list:
        mail_dataframe[count_repl_patt.pattern_cat]=[]
    for count_patt in count_content_list:
        mail_dataframe[count_patt.pattern_cat]=[]
    mail_dataframe['all words']=[]
    
    return mail_dataframe
    

In [272]:
def process_mail(raw_text, mail_dataframe, column_name_list, spam=True):
    heading, content = separate_header_content(raw_text)
    heading_attributes = process_heading(heading)
    content_attributes = process_content(content)
    mail_attributes =  [int(spam)] + heading_attributes + content_attributes
    #len(mail_dataframe.index)
    dict2add = dict(zip(column_name_list, mail_attributes))
    #print(dict2add)
    return mail_dataframe.append(dict2add, ignore_index=True)
    #mail_dataframe.loc[] = mail_attributes

In [273]:
def extract_data_from_file(spam_folders_path, ham_folders_path, mail_dataframe=None):
    if mail_dataframe == None:
        # if no mail has been processed and saved before create dataframe
        mail_dataframe = create_dataframe_cat()
        
    column_name_list = list(mail_dataframe.columns)
    for folder_path in spam_folders_path:
        under_path = folder_path + os.listdir(folder_path)[0] + "/"
        file_list = os.listdir(under_path)
        number2extract = 10*(len(file_list))//100
        print(number2extract)
        for file in file_list[len(file_list)-number2extract:]:
            filepath = under_path + file
            if (file!="cmds"):
                extraction, text = extract_txt(filepath)
                if (extraction == True):
                    mail_dataframe = process_mail(text, mail_dataframe,column_name_list, spam=True)
                else:
                    print(text)
                #return 0
            
    for folder_path in ham_folders_path:
        under_path = folder_path + os.listdir(folder_path)[0] + "/"
        file_list = os.listdir(under_path)
        number2extract = 7*(len(file_list))//100
        print(number2extract)
        for file in file_list[len(file_list)-number2extract:]:
            filepath = under_path + file
            if (file!="cmds"):
                extraction, text = extract_txt(filepath)
                if (extraction == True):
                    mail_dataframe = process_mail(text, mail_dataframe,column_name_list, spam=False)
                else:
                    print(text)
                #return 0
    return mail_dataframe

In [274]:
mail_dataframe = extract_data_from_file(spam_folders_path, ham_folders_path, mail_dataframe=None)

139
50
50
98
17
175
178
17


In [275]:
mail_dataframe

,spam,from,subject,hour,content-type,importance,x-spam,x-*,html,url,mail,price,> count,? or !,content_attr,Number count,Majuscule count,all words
0,1.0,"[17, 0, mail, com]","[5, 44, 4, 0, 3, [make, love, tonight, wDFrsxqw]]",18.0,text/html,NaN,NaN,5.0,21.0,1.0,0.0,0.0,0.0,0.0,"[6, 3.6667, 5]",0.0,0.0,"[open, click, here, see, more]"
1,1.0,"[23, 1, ct2.consumertoday, net]","[4, 48, 1, 0, 0, [Complimentary, Thermal, Coff...",19.0,multipart/alternative,NaN,NaN,2.0,0.0,2.0,0.0,4.0,0.0,2.0,"[177, 5.2147, 63]",11.0,58.0,"[Content, Type, text, plain, charset, ascii, C..."
2,1.0,"[13, 2, address, com]","[3, 44, 0, 0, 1, [Your, application, below, Ex...",18.0,text/html,NaN,NaN,1.0,121.0,2.0,1.0,1.0,1.0,9.0,"[184, 4.8641, 47]",2.0,84.0,"[Your, application, for, the, Grant, below, Re..."
3,1.0,"[9, 4, hotmail, com]","[4, 37, 3, 0, 0, [Incredible, Cash, Generating...",12.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,"[124, 4.3145, 14]",3.0,31.0,"[Greetings, Like, most, have, been, burned, se..."
4,1.0,"[16, 0, TomGreen, com]","[9, 74, 0, 0, 0, [Free, Adult, DVDs, purchase,...",18.0,text/plain,NaN,NaN,1.0,0.0,2.0,1.0,1.0,1.0,15.0,"[165, 4.9636, 26]",3.0,104.0,"[Adult, Online, Super, Store, Shhhhhh, You, ju..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,0.0,"[29, 0, tribute, ca]","[4, 31, 0, 0, 2, [Tribute, MovieMail, Vol]]",0.0,multipart/alternative,NaN,NaN,0.0,2025.0,2.0,0.0,0.0,0.0,15.0,"[2997, 3.6994, 27]",1432.0,498.0,"[This, multi, part, message, MIME, format, _Ne..."
720,0.0,"[35, 0, DivX.at.krieger.mailshell, com]","[11, 64, 0, 0, 0, [DivX, News, Battle, the, Pl...",19.0,text/plain,NaN,NaN,10.0,14.0,3.0,0.0,1.0,0.0,4.0,"[768, 4.7044, 14]",18.0,226.0,"[DivX, News, Number, What, Hot, the, World, Di..."
721,0.0,"[61, 9, newsletter.online, com]","[6, 42, 0, 0, 0, [Where, TabletPC, will, succe...",6.0,text/html,NaN,NaN,2.0,1009.0,1.0,2.0,0.0,0.0,6.0,"[855, 5.1427, 19]",33.0,372.0,"[Tech, Update, Today, VITAL, SIGNS, FOR, JULY,..."
722,0.0,"[10, 0, mountainviewdata, com]","[0, 19, 0, 0, 0, [deadlock, problem]]",17.0,text/plain; charset=us-ascii,NaN,NaN,10.0,0.0,0.0,0.0,0.0,0.0,1.0,"[127, 4.9449, 15]",6.0,105.0,"[XFS, developers, When, studying, xfs, code, f..."


Save dataframe to csv
---

In [276]:
mail_dataframe.to_csv(os.path.abspath(os.getcwd()) + "/mail_extract.csv", index = False)

Split categories in function of spam or not 
----

In [277]:
def split_ham_spam_dataframe(mail_dataframe, cat2split):
    spam = mail_dataframe.loc[mail_dataframe["spam"] == 1.0][cat2split]
    ham = mail_dataframe.loc[mail_dataframe["spam"] == 0.0][cat2split] 
    ham.reset_index(drop=True, inplace=True)
    
    return (spam, ham)

In [278]:
# Process subject
spam_subject, ham_subject = split_ham_spam_dataframe(mail_dataframe, "subject")
# Process content
spam_content, ham_content = split_ham_spam_dataframe(mail_dataframe, "all words")

Search top numbers_of_words of most used words
---

In [279]:
def search_subject_redondant_word(subject_dataframe, number_of_words):
    all_word_spam = []
    for i in range(subject_dataframe.shape[0]):
        all_word_spam += subject_dataframe[i][5]
    spam_most_used_words = Counter(map(str.lower, all_word_spam)).most_common(number_of_words)
    spam_most_used_words = [(i[0], round(i[1]/len(all_word_spam), 4)) for i in spam_most_used_words]
    return spam_most_used_words

In [280]:
spam_subject_most_used_words = search_subject_redondant_word(spam_subject, 20)
spam_subject_most_used_words

[('your', 0.0348),
 ('the', 0.0196),
 ('free', 0.0187),
 ('you', 0.0187),
 ('for', 0.0161),
 ('home', 0.0134),
 ('adv', 0.0116),
 ('from', 0.0107),
 ('best', 0.008),
 ('with', 0.008),
 ('help', 0.008),
 ('mortgage', 0.0071),
 ('get', 0.0071),
 ('and', 0.0071),
 ('online', 0.0071),
 ('money', 0.0063),
 ('work', 0.0063),
 ('make', 0.0054),
 ('business', 0.0054),
 ('low', 0.0054)]

In [281]:
ham_subject_most_used_words = search_subject_redondant_word(ham_subject, 20)
ham_subject_most_used_words

[('ilug', 0.0235),
 ('for', 0.022),
 ('the', 0.022),
 ('razor', 0.0141),
 ('users', 0.0094),
 ('with', 0.0094),
 ('new', 0.0094),
 ('spambayes', 0.0094),
 ('satalk', 0.0089),
 ('zzzzteana', 0.0084),
 ('use', 0.0075),
 ('and', 0.007),
 ('perl', 0.0061),
 ('was', 0.0061),
 ('spam', 0.0056),
 ('sadev', 0.0052),
 ('apt', 0.0052),
 ('spamassassin', 0.0047),
 ('stories', 0.0042),
 ('lockergnome', 0.0042)]

In [282]:
def search_content_redondant_word(word_dataframe, number_of_words):
    all_word_spam = []
    for i in range(word_dataframe.shape[0]):
        all_word_spam += word_dataframe[i]
    spam_most_used_words = Counter(map(str.lower, all_word_spam)).most_common(number_of_words)
    spam_most_used_words = [(i[0], round(i[1]/len(all_word_spam), 4)) for i in spam_most_used_words]
    return spam_most_used_words

In [283]:
spam_most_used_words = search_content_redondant_word(spam_content, 100)
spam_most_used_words

[('the', 0.0344),
 ('you', 0.0292),
 ('and', 0.0259),
 ('for', 0.0156),
 ('your', 0.0143),
 ('this', 0.0126),
 ('nbsp', 0.0122),
 ('that', 0.0081),
 ('with', 0.0076),
 ('are', 0.0073),
 ('will', 0.0072),
 ('email', 0.0065),
 ('from', 0.0065),
 ('our', 0.0063),
 ('have', 0.0063),
 ('free', 0.0062),
 ('not', 0.0057),
 ('all', 0.0047),
 ('can', 0.0046),
 ('money', 0.004),
 ('here', 0.0038),
 ('get', 0.0037),
 ('business', 0.0037),
 ('more', 0.0036),
 ('out', 0.0036),
 ('list', 0.0034),
 ('click', 0.0033),
 ('please', 0.0032),
 ('mail', 0.0031),
 ('one', 0.0031),
 ('only', 0.0031),
 ('receive', 0.003),
 ('now', 0.003),
 ('time', 0.0029),
 ('order', 0.0028),
 ('any', 0.0027),
 ('how', 0.0026),
 ('new', 0.0025),
 ('people', 0.0025),
 ('cfont', 0.0025),
 ('ffont', 0.0025),
 ('information', 0.0024),
 ('content', 0.0024),
 ('just', 0.0024),
 ('send', 0.0023),
 ('name', 0.0023),
 ('address', 0.0022),
 ('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
  0.0022),
 ('

In [284]:
ham_most_used_words = search_content_redondant_word(ham_content, 100)
ham_most_used_words

[('the', 0.054),
 ('and', 0.0241),
 ('that', 0.0146),
 ('for', 0.012),
 ('you', 0.0095),
 ('this', 0.0087),
 ('nbsp', 0.0085),
 ('with', 0.0078),
 ('from', 0.0064),
 ('not', 0.0058),
 ('have', 0.0054),
 ('but', 0.005),
 ('are', 0.0048),
 ('was', 0.0046),
 ('your', 0.0045),
 ('font', 0.0042),
 ('can', 0.004),
 ('all', 0.0039),
 ('has', 0.0036),
 ('they', 0.0034),
 ('list', 0.0034),
 ('one', 0.0032),
 ('linux', 0.0032),
 ('will', 0.0031),
 ('more', 0.0031),
 ('there', 0.003),
 ('about', 0.003),
 ('what', 0.0029),
 ('just', 0.0029),
 ('color', 0.0028),
 ('which', 0.0026),
 ('use', 0.0026),
 ('out', 0.0025),
 ('now', 0.0024),
 ('some', 0.0024),
 ('get', 0.0023),
 ('when', 0.0022),
 ('like', 0.0022),
 ('new', 0.0022),
 ('would', 0.0022),
 ('their', 0.0022),
 ('been', 0.0021),
 ('other', 0.0021),
 ('mail', 0.0021),
 ('only', 0.002),
 ('family', 0.002),
 ('message', 0.0019),
 ('then', 0.0019),
 ('time', 0.0019),
 ('wrote', 0.0019),
 ('size', 0.0019),
 ('than', 0.0019),
 ('its', 0.0018),
 ('te

Stemming, concatenate and delete redundancy
----
Stemming = took the root of the word -> in order to merge similar words (ex: "use" "user" "uses") and Concatenate lists from both spam and ham most used words and suppress doublons

In [285]:
def stem_concatenate_most_used_word(spam_most_used, ham_most_used):
    ps = PorterStemmer()
    
    list_most_used_word = [ps.stem(i[0]) for i in spam_most_used]
    list_most_used_word = list_most_used_word + [ps.stem(i[0]) for i in ham_most_used]
    list_most_used_word = list(set(list_most_used_word))
    
    return list_most_used_word   

In [286]:
# Process subject
subject_most_used_words = stem_concatenate_most_used_word(spam_subject_most_used_words, ham_subject_most_used_words)
# Process content
content_most_used_words = stem_concatenate_most_used_word(spam_most_used_words, ham_most_used_words)

In [287]:
old_most_used_word_subject = [
'perl',
 'free',
 'user',
 'insur',
 'from',
 'and',
 'for',
 'problem',
 'get',
 'sadev',
 'new',
 'use',
 'razor',
 'spam',
 'email',
 'you',
 'spambay',
 'now',
 'ilug',
 'best',
 'satalk',
 'with',
 'adv',
 'the',
 'market',
 'big',
 'mortgag',
 'apt',
 'your',
 'ouch',
 'spamassassin',
 'wa',
 'life',
 'onlin']

In [306]:
subject_intersection = set(old_most_used_word_subject).intersection(subject_most_used_words)
subject_intersection = list(subject_intersection)

['perl',
 'free',
 'user',
 'from',
 'and',
 'for',
 'get',
 'sadev',
 'new',
 'use',
 'razor',
 'spam',
 'you',
 'spambay',
 'ilug',
 'best',
 'satalk',
 'with',
 'adv',
 'the',
 'mortgag',
 'apt',
 'your',
 'spamassassin',
 'wa',
 'onlin']

In [230]:
most_used_word_subject = [
'perl',
 'free',
 'user',
 'from',
 'and',
 'for',
 'get',
 'sadev',
 'new',
 'use',
 'razor',
 'spam',
 'you',
 'spambay',
 'ilug',
 'best',
 'satalk',
 'with',
 'adv',
 'the',
 'mortgag',
 'apt',
 'your',
 'spamassassin',
 'wa',
 'onlin']

[('the', 0.054),
 ('and', 0.0241),
 ('that', 0.0146),
 ('for', 0.012),
 ('you', 0.0095),
 ('this', 0.0087),
 ('nbsp', 0.0085),
 ('with', 0.0078),
 ('from', 0.0064),
 ('not', 0.0058),
 ('have', 0.0054),
 ('but', 0.005),
 ('are', 0.0048),
 ('was', 0.0046),
 ('your', 0.0045),
 ('font', 0.0042),
 ('can', 0.004),
 ('all', 0.0039),
 ('has', 0.0036),
 ('they', 0.0034),
 ('list', 0.0034),
 ('one', 0.0032),
 ('linux', 0.0032),
 ('will', 0.0031),
 ('more', 0.0031),
 ('there', 0.003),
 ('about', 0.003),
 ('what', 0.0029),
 ('just', 0.0029),
 ('color', 0.0028),
 ('which', 0.0026),
 ('use', 0.0026),
 ('out', 0.0025),
 ('now', 0.0024),
 ('some', 0.0024),
 ('get', 0.0023),
 ('when', 0.0022),
 ('like', 0.0022),
 ('new', 0.0022),
 ('would', 0.0022),
 ('their', 0.0022),
 ('been', 0.0021),
 ('other', 0.0021),
 ('mail', 0.0021),
 ('only', 0.002),
 ('family', 0.002),
 ('message', 0.0019),
 ('then', 0.0019),
 ('time', 0.0019),
 ('wrote', 0.0019),
 ('size', 0.0019),
 ('than', 0.0019),
 ('its', 0.0018),
 ('te

In [288]:
old_most_used_word_content = [
'phone',
 'had',
 'program',
 'for',
 'there',
 'type',
 'get',
 'should',
 'not',
 'busi',
 'new',
 'them',
 'think',
 'web',
 'about',
 'world',
 'govern',
 'just',
 'date',
 'with',
 'the',
 'darial',
 'other',
 'url',
 'see',
 'thi',
 'receiv',
 'size',
 'wa',
 'pleas',
 'becaus',
 'need',
 'state',
 'are',
 'click',
 'compani',
 'linux',
 'today',
 'even',
 'insur',
 'who',
 'could',
 'from',
 'too',
 'text',
 'have',
 'they',
 'some',
 'unit',
 'one',
 'also',
 'grant',
 'year',
 'would',
 'home',
 'peopl',
 'use',
 'same',
 'email',
 'you',
 'ani',
 'make',
 'those',
 'over',
 'cfont',
 'out',
 'much',
 'inform',
 'their',
 'these',
 'your',
 'spamassassin',
 'into',
 'list',
 'kingdom',
 'more',
 'order',
 'nbsp',
 'life',
 'wrote',
 'don',
 'work',
 'call',
 'way',
 'free',
 'mail',
 'and',
 'can',
 'then',
 'ha',
 'ffont',
 'face',
 'send',
 'hi',
 'now',
 'what',
 'time',
 'remov',
 'here',
 'been',
 'our',
 'net',
 'internet',
 'than',
 'rpm',
 'onli',
 'form',
 'how',
 'it',
 'transfer',
 'will',
 'user',
 'right',
 'which',
 'want',
 'know',
 'were',
 'but',
 'most',
 'that',
 'name',
 'content',
 'money',
 'well',
 'all',
 'address',
 'may',
 'find',
 'messag',
 'color',
 'when',
 'help',
 'enenkio',
 'island',
 'like']

In [298]:
content_intersection = set(old_most_used_word_content).intersection(content_most_used_words)
most_used_word_content = list(content_intersection)

In [299]:
most_used_word_content = [
'phone',
 'had',
 'for',
 'there',
 'type',
 'get',
 'should',
 'not',
 'busi',
 'new',
 'them',
 'think',
 'web',
 'about',
 'world',
 'govern',
 'just',
 'date',
 'with',
 'the',
 'other',
 'url',
 'see',
 'thi',
 'receiv',
 'size',
 'wa',
 'pleas',
 'becaus',
 'need',
 'are',
 'click',
 'compani',
 'linux',
 'even',
 'who',
 'could',
 'from',
 'text',
 'have',
 'they',
 'some',
 'one',
 'also',
 'grant',
 'would',
 'home',
 'peopl',
 'use',
 'email',
 'you',
 'ani',
 'make',
 'over',
 'cfont',
 'out',
 'much',
 'inform',
 'their',
 'your',
 'spamassassin',
 'into',
 'list',
 'more',
 'order',
 'nbsp',
 'wrote',
 'don',
 'work',
 'call',
 'way',
 'free',
 'mail',
 'and',
 'can',
 'then',
 'ha',
 'ffont',
 'send',
 'face',
 'now',
 'what',
 'time',
 'remov',
 'here',
 'been',
 'our',
 'internet',
 'than',
 'onli',
 'form',
 'how',
 'it',
 'will',
 'user',
 'which',
 'want',
 'know',
 'were',
 'but',
 'most',
 'that',
 'name',
 'content',
 'money',
 'well',
 'all',
 'address',
 'may',
 'messag',
 'color',
 'when',
 'help',
 'like']

['phone',
 'had',
 'for',
 'there',
 'type',
 'get',
 'should',
 'not',
 'busi',
 'new',
 'them',
 'think',
 'web',
 'about',
 'world',
 'govern',
 'just',
 'date',
 'with',
 'the',
 'other',
 'url',
 'see',
 'thi',
 'receiv',
 'size',
 'wa',
 'pleas',
 'becaus',
 'need',
 'are',
 'click',
 'compani',
 'linux',
 'even',
 'who',
 'could',
 'from',
 'text',
 'have',
 'they',
 'some',
 'one',
 'also',
 'grant',
 'would',
 'home',
 'peopl',
 'use',
 'email',
 'you',
 'ani',
 'make',
 'over',
 'cfont',
 'out',
 'much',
 'inform',
 'their',
 'your',
 'spamassassin',
 'into',
 'list',
 'more',
 'order',
 'nbsp',
 'wrote',
 'don',
 'work',
 'call',
 'way',
 'free',
 'mail',
 'and',
 'can',
 'then',
 'ha',
 'ffont',
 'send',
 'face',
 'now',
 'what',
 'time',
 'remov',
 'here',
 'been',
 'our',
 'internet',
 'than',
 'onli',
 'form',
 'how',
 'it',
 'will',
 'user',
 'which',
 'want',
 'know',
 'were',
 'but',
 'most',
 'that',
 'name',
 'content',
 'money',
 'well',
 'all',
 'address',
 'may',